In [1]:
import torch
import torch.nn as nn

In [7]:
lstm = nn.LSTM(input_size=100, hidden_size=20, num_layers=4)
X = torch.randn(10, 3, 100)       # 一个句子10个单词，送进去3条句子，每个单词用一个100维的vector表示
out, (h_n, c_n) = lstm(X)
print(out.shape, h_n.shape, c_n.shape)

torch.Size([10, 3, 20]) torch.Size([4, 3, 20]) torch.Size([4, 3, 20])


tensor([[[ 1.8368e-02, -4.1126e-04, -5.8868e-02, -3.2964e-02,  1.4120e-02,
          -4.7668e-02, -1.5838e-02,  4.8270e-02,  2.2404e-02, -4.1941e-02,
          -1.9616e-02, -1.7813e-02, -8.6861e-02, -2.7224e-02,  6.8505e-02,
           3.8289e-02, -8.8780e-02, -4.1401e-02,  3.3371e-02, -1.4062e-02],
         [ 1.8224e-02, -1.3267e-05, -5.9060e-02, -3.2871e-02,  1.3553e-02,
          -4.6836e-02, -1.6258e-02,  4.8142e-02,  2.2594e-02, -4.1784e-02,
          -1.9516e-02, -1.7391e-02, -8.6209e-02, -2.6872e-02,  6.8792e-02,
           3.7712e-02, -8.8662e-02, -4.1273e-02,  3.3494e-02, -1.4386e-02],
         [ 1.8635e-02, -1.0398e-05, -5.7922e-02, -3.3481e-02,  1.5866e-02,
          -4.6833e-02, -1.5050e-02,  4.9025e-02,  2.1558e-02, -4.3071e-02,
          -1.9062e-02, -1.7835e-02, -8.5562e-02, -2.6165e-02,  6.7884e-02,
           3.6023e-02, -8.9319e-02, -4.1004e-02,  3.4693e-02, -1.3583e-02]],

        [[ 2.9072e-02,  2.0472e-03, -7.2441e-02, -3.7136e-02,  1.0711e-02,
          -7.0736e-0

In [8]:
cell = nn.LSTMCell(input_size=100, hidden_size=20)
x = torch.randn(10, 3, 100)
h = torch.zeros(3,20)
c = torch.zeros(3,20)

for xt in x:
    h, c = cell(xt, [h,c])

print(h.shape, c.shape)

torch.Size([3, 20]) torch.Size([3, 20])


In [9]:
cell1 = nn.LSTMCell(input_size=100, hidden_size=30)
cell2 = nn.LSTMCell(input_size=30, hidden_size=20)

x = torch.randn(10, 3, 100)

h1 = torch.zeros(3, 30)
c1 = torch.zeros(3, 30)

h2 = torch.zeros(3,20)
c2 = torch.zeros(3,20)

for xt in x:
    h1, c1 = cell1(xt, [h1,c1])
    h2, c2 = cell2(h1, [h2,c2])

print(h2.shape, c2.shape)

torch.Size([3, 20]) torch.Size([3, 20])


In [10]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [16]:
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)    # 实际上是一句话

word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}

n_class = len(word2idx)
max_len = len(sentence.split())

n_hidden = 5

In [21]:
max_len

21

In [34]:
def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split()

    for i in range(max_len-1):
        input = [word2idx[w] for w in words[:(i+1)]]     # (i+1)位置取不到
        input = input + [0] * (max_len - len(input))
        print(np.eye(n_class)[input].shape)
        print(len(input))
        input_batch.append(np.eye(n_class)[input])
        target = word2idx[words[i+1]]
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_data(sentence)
dataset = Data.TensorDataset(input_batch, target_batch)
dataloader = Data.DataLoader(dataset, 16 ,True)     # 中间参数是batch_size

(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21
(21, 19)
21


In [35]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        # input_size表示每个词用input_size维的vector表示
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        # input: [max_len, batch_size, n_class]
        input = X.transpose(0,1)
        # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        hidden_state = torch.randn(1*2, batch_size, n_hidden)
        # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1*2, batch_size, n_hidden)

        outputs, (_,_) = self.lstm(input, (hidden_state, cell_state))
        # outputs: [batch_size, n_hidden * 2]
        outputs = outputs[-1]
        # model: [batch_size, n_class]
        model = self.fc(outputs)

        return model

modol = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modol.parameters(), lr=0.001)